In [1]:
# pip install torch

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("C:/Users\Javi\PycharmProjects\pythonProject_XAI\XAI-grupazo"))

# Any results you write to the current directory are saved as output.

['.git', '.ipynb_checkpoints', 'net.pth', 'new_net.pth', 'new_submission.csv', 'README.md', 'submission.csv', 'test.csv', 'titanic-pytorch-Copy1.ipynb', 'titanic-pytorch.ipynb', 'titanic_lime.ipynb', 'train.csv']


In [3]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
# Se importa la biblioteca PyTorch para trabajar con redes neuronales y tensores.
import torch
# Se importa el módulo nn de PyTorch, que proporciona las herramientas para definir y entrenar redes neuronales.
import torch.nn as nn
# Se importa el módulo functional de PyTorch, que contiene varias funciones útiles para trabajar con DNN, como funciones de activación.
import torch.nn.functional as F
# Se importa la función shuffle del módulo utils de scikit-learn. Esta función se utiliza para mezclar los datos de entrenamiento en cada época.
from sklearn.utils import shuffle
# Se importa Variable de PyTorch. Se utiliza para envolver los tensores y habilitar el cálculo automático de gradientes durante el entrenamiento.
from torch.autograd import Variable
import csv

In [4]:
dataset = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

In [5]:
# Está creando una lista llamada dataset_title que contiene los títulos de las personas en la columna 'Name' del DataFrame dataset. 
# Se divide cada nombre en partes con comas y puntos, se toma la segunda parte (índice 1) antes de eliminar los espacios adicionales con strip().
dataset_title = [i.split(',')[1].split('.')[0].strip() for i in dataset['Name']]

# Crea una nueva columna llamada 'Title' en el DataFrame dataset y la llena con los valores de la lista dataset_title.
dataset['Title'] = pd.Series(dataset_title)

# Devuelve el recuento de cada valor en la columna 'Title', lo que te da una idea de cuántas veces aparece cada título en el conjunto de datos.
dataset['Title'].value_counts()

# Reemplaza títulos poco comunes en la columna 'Title' del DataFrame dataset con el valor 'Rare'. 
# Los títulos específicos que se reemplazan son: 
# 'Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'.
dataset['Title'] = dataset['Title'].replace(['Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')

# Las siguientes líneas siguen un proceso similar para el DataFrame X_test, creando la columna 'Title'. 
# Cuenta los valores únicos y reemplazandolos títulos poco comunes por 'Rare'.
dataset_title = [i.split(',')[1].split('.')[0].strip() for i in X_test['Name']]
X_test['Title'] = pd.Series(dataset_title)
X_test['Title'].value_counts()
X_test['Title'] = X_test['Title'].replace(['Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')

# calcula el tamaño de la familia para cada entrada en el DataFrame dataset. 
# Suma las columnas 'SibSp' (nº de hermanos/cónyuges a bordo) y 'Parch' (nº de padres/hijos a bordo) y le suma 1 para incluir al pasajero mismo.
dataset['FamilyS'] = dataset['SibSp'] + dataset['Parch'] + 1

# De manera similar, se calcula el tamaño de la familia para cada entrada en el DataFrame X_test.
X_test['FamilyS'] = X_test['SibSp'] + X_test['Parch'] + 1

In [6]:
# La función family(x) toma un valor x como entrada y asigna una categoría a ese valor en función de las condiciones establecidas:
def family(x):
    
    # Si x es menor que 2, se asigna la categoría 'Single' (soltero).
    if x < 2:
        return 'Single'
    
    # Si x es igual a 2, se asigna la categoría 'Couple' (pareja).
    elif x == 2:
        return 'Couple'
    
    # Si x es menor o igual a 4, se asigna la categoría 'InterM' (intermedia).
    elif x <= 4:
        return 'InterM'
    
    # Si ninguna de las condiciones anteriores se cumple, se asigna la categoría 'Large' (grande).
    else:
        return 'Large'

# Asigna una categoría a cada valor en la columna 'FamilyS' en "dataset" y "X_test" .
# Es una utilidad para analizar el tamaño de la familia y agrupar los datos en categorías más significativas.
dataset['FamilyS'] = dataset['FamilyS'].apply(family)
X_test['FamilyS'] = X_test['FamilyS'].apply(family)

El código está relacionado con el manejo de valores faltantes (NaN) en diferentes columnas de los DataFrames dataset y X_test. 

dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace=True): En la columna 'Embarked' del DataFrame dataset, se están 
reemplazando los valores faltantes con el valor más común (moda) de esa columna. La función mode()[0] devuelve el valor más común y el 
parámetro inplace=True asegura que los cambios se realicen en el DataFrame original.

X_test['Embarked'].fillna(X_test['Embarked'].mode()[0], inplace=True): De manera similar al paso anterior, aquí se reemplazan los valores faltantes
en la columna 'Embarked' del DataFrame X_test con la moda de esa columna.

dataset['Age'].fillna(dataset['Age'].median(), inplace=True): En la columna 'Age' del DataFrame dataset, se están reemplazando los valores 
faltantes con la mediana de esa columna. La función median() calcula la mediana y el parámetro inplace=True asegura que los cambios se 
realicen en el DataFrame original.

X_test['Age'].fillna(X_test['Age'].median(), inplace=True): De manera similar al paso anterior, aquí se reemplazan los valores faltantes en la 
columna 'Age' del DataFrame X_test con la mediana de esa columna.

X_test['Fare'].fillna(X_test['Fare'].median(), inplace=True): En la columna 'Fare' del DataFrame X_test, se están reemplazando los valores faltantes
con la mediana de esa columna. Esto se realiza solo en el DataFrame X_test.

In [7]:

dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace=True)

X_test['Embarked'].fillna(X_test['Embarked'].mode()[0], inplace=True)

dataset['Age'].fillna(dataset['Age'].median(), inplace=True)

X_test['Age'].fillna(X_test['Age'].median(), inplace=True)

X_test['Fare'].fillna(X_test['Fare'].median(), inplace=True)

dataset = dataset.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1): En el DataFrame dataset, se están eliminando las 
columnas 'PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch' y 'Ticket'. El parámetro axis=1 indica que se deben eliminar columnas en lugar de filas.

X_test_passengers = X_test['PassengerId']: Se está creando una nueva variable X_test_passengers que contiene los valores de la columna 
'PassengerId' del DataFrame X_test.

X_test = X_test.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1): En el DataFrame X_test, se están eliminando las mismas 
columnas ('PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket') que se eliminaron en el DataFrame dataset.

In [8]:

dataset = dataset.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1)

X_test_passengers = X_test['PassengerId']

X_test = X_test.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1)

In [9]:
dataset.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilyS
0,0,3,male,22.0,7.2500,S,Mr,Couple
1,1,1,female,38.0,71.2833,C,Mrs,Couple
2,1,3,female,26.0,7.9250,S,Miss,Single
3,1,1,female,35.0,53.1000,S,Mrs,Couple
4,0,3,male,35.0,8.0500,S,Mr,Single


X_train = dataset.iloc[:, 1:9].values: Aquí se está extrayendo una porción de dataset que incluye todas las filas y las columnas de la 1 a la 9 
(excluyendo la columna 0). Los valores se asignan a la variable X_train.

Y_train = dataset.iloc[:, 0].values: Se extraen los valores de la primera columna (columna 0) de dataset y se asignan a la variable Y_train. 
Esto se hace porque la columna 0 contiene la variable objetivo (la etiqueta o resultado que se desea predecir).

X_test = X_test.values: Aquí se extraen los valores del DataFrame X_test y se asignan a la variable X_test. Esto se hace para preparar los datos 
de prueba para su uso posterior.

In [10]:

X_train = dataset.iloc[:, 1:9].values

Y_train = dataset.iloc[:, 0].values

X_test = X_test.values

labelencoder_X_1 = LabelEncoder(): Se crea una instancia de LabelEncoder y se asigna a la variable labelencoder_X_1.

X_train[:, 1] = labelencoder_X_1.fit_transform(X_train[:, 1]): La columna 1 de X_train (la cual contiene etiquetas categóricas) se convierte a valores
numéricos utilizando el labelencoder_X_1. Los valores convertidos se asignan de nuevo a la columna 1 de X_train.

En los siguientes pasos se repite para las columnas 4, 5 y 6 de X_train.

labelencoder_X_2 = LabelEncoder(): Se crea otra instancia de LabelEncoder y se asigna a la variable labelencoder_X_2.

Se aplican idénticos cambios a las columnas correspondientes de X_test en lugar de X_train.


Este código utiliza LabelEncoder para convertir las etiquetas categóricas en las columnas seleccionadas de X_train y X_test en valores numéricos 
para que los algoritmos de aprendizaje automático puedan trabajar con características categóricas en lugar de texto.

In [11]:
# Converting the remaining labels to numbers
labelencoder_X_1 = LabelEncoder()
X_train[:, 1] = labelencoder_X_1.fit_transform(X_train[:, 1])
X_train[:, 4] = labelencoder_X_1.fit_transform(X_train[:, 4])
X_train[:, 5] = labelencoder_X_1.fit_transform(X_train[:, 5])
X_train[:, 6] = labelencoder_X_1.fit_transform(X_train[:, 6])

labelencoder_X_2 = LabelEncoder()
X_test[:, 1] = labelencoder_X_2.fit_transform(X_test[:, 1])
X_test[:, 4] = labelencoder_X_2.fit_transform(X_test[:, 4])
X_test[:, 5] = labelencoder_X_2.fit_transform(X_test[:, 5])
X_test[:, 6] = labelencoder_X_2.fit_transform(X_test[:, 6])

In [12]:

# Índices de las columnas categóricas
categorical_columns = [0, 1, 4, 5, 6]

# Crear el transformador de columnas
column_transformer = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(), categorical_columns)],
    remainder='passthrough'
)

# Aplicar la transformación a X_train y X_test
X_train = column_transformer.fit_transform(X_train)
X_test = column_transformer.transform(X_test)


In [13]:
X_test[0].shape

(19,)

In [14]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)

In [15]:
'''
Se utiliza la función train_test_split de scikit-learn para dividir los datos de entrenamiento (X_train y Y_train) 
en conjuntos de entrenamiento y validación.
'''
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.1)

In [16]:
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

(801, 19) (90, 19) (801,) (90,)


In [17]:
'''
En este código se define una red neuronal utilizando la biblioteca PyTorch.
'''

# Se define una clase llamada Net que hereda de la clase nn.Module, que es la base para definir redes neuronales en PyTorch.
class Net(nn.Module):
    
    # Se define el método constructor, que se llama al crear una instancia de la clase Net. Aquí se definen las capas de la red neuronal.
    def __init__(self):
        
        # super(Net, self).__init__(): Se llama al método __init__ de la clase base nn.Module para inicializar la clase.
        super(Net, self).__init__()
        
        # Se define la capa completamente conectada fc1 con 19 entradas y 270 neuronas.
        self.fc1 = nn.Linear(19, 270)
        
        # Se define la capa completamente conectada fc2 con 270 entradas y 2 neuronas, que corresponde a las clases de salida.
        self.fc2 = nn.Linear(270, 2)
        
    # Se define el método forward, que especifica cómo los datos se propagan a través de la red neuronal.
    def forward(self, x):
        
        # Los datos de entrada x se propagan a través de la capa fc1.
        x = self.fc1(x)
        
        # Se aplica una capa de dropout con una probabilidad de desactivación del 10% a los datos x.
        # Esto ayuda a prevenir el sobreajuste en la red neuronal.
        x = F.dropout(x, p=0.1)
        
        # Se aplica la función de activación ReLU a los datos x.
        x = F.relu(x)
        
        # Los datos x se propagan a través de la capa fc2.
        x = self.fc2(x)
        
        # Se aplica la función de activación sigmoide a los datos x para obtener las probabilidades de clase.

# C:\Users\Javi\anaconda3\lib\site-packages\torch\nn\functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. 
# Use torch.sigmoid instead. warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.)
        x = torch.softmax(x, dim=1)

        return x


    def predict(self, x):
        
        if isinstance(x, np.ndarray):
            
            x = torch.from_numpy(x).float()
            
        elif isinstance(x, torch.Tensor):
            
            x = x.float()

        # Asegúrate de que los datos de entrada tengan la forma correcta
        x = x.view(x.size(0), -1)

        # Propagación hacia adelante de la red neuronal
        output = self.forward(x)

        # Convierte las probabilidades de clase a etiquetas (0 o 1)
        _, predicted_labels = torch.max(output, 1)
    
        return predicted_labels.numpy()

# Se crea una instancia de la clase Net y se asigna a la variable net.
new_net = Net()

In [18]:
'''   
    def predict(self, x):
        
        
        # Aplica la propagación hacia adelante de la red neuronal
        output = self.forward(x)
        # Convierte las probabilidades de clase a etiquetas (0 o 1)
        _, predicted_labels = torch.max(output, 1)
        return predicted_labels
'''

'   \n    def predict(self, x):\n        \n        \n        # Aplica la propagación hacia adelante de la red neuronal\n        output = self.forward(x)\n        # Convierte las probabilidades de clase a etiquetas (0 o 1)\n        _, predicted_labels = torch.max(output, 1)\n        return predicted_labels\n'

In [19]:
'''
batch_size = 50: Se define el tamaño del lote, que es el número de muestras que se utilizan para calcular los gradientes y actualizar los pesos en
cada paso de entrenamiento. En este caso, se establece en 50, lo que significa que se utilizarán 50 muestras a la vez durante el entrenamiento.

num_epochs = 50: Se define el número de épocas, que es la cantidad de veces que el modelo pasará por todo el conjunto de entrenamiento.
En este caso, se establece en 50, lo que significa que el modelo se entrenará durante 50 pasadas completas por el conjunto de entrenamiento.

learning_rate = 0.01: Se define la tasa de aprendizaje, que controla el tamaño de los pasos de actualización de los pesos durante el 
entrenamiento. En este caso, se establece en 0.01, lo que significa que los pesos se actualizarán en incrementos pequeños.

batch_no = len(x_train) // batch_size: Se calcula el número de lotes en cada época dividiendo el número total de muestras de entrenamiento 
(len(x_train)) por el tamaño del lote (batch_size). Esto proporciona el número de veces que se iterará sobre los lotes durante cada época.
'''

batch_size = 25
num_epochs = 100
learning_rate = 0.001
batch_no = len(x_train) // batch_size

In [20]:
'''
criterion = nn.CrossEntropyLoss(): Se crea una instancia de la clase CrossEntropyLoss de PyTorch. Esta función de pérdida se utiliza para 
problemas de clasificación multiclase. En este caso, se utiliza para calcular la pérdida entre las probabilidades de salida de la red neuronal y las
etiquetas verdaderas.

optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate): Se crea una instancia del optimizador Adam utilizando los parámetros de la red 
neuronal (net.parameters()) y la tasa de aprendizaje (learning_rate). El optimizador Adam es un algoritmo de optimización popular utilizado para 
ajustar los pesos de la red neuronal durante el entrenamiento. Se encarga de actualizar los pesos de acuerdo con los gradientes calculados a 
partir de la función de pérdida.
'''

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(new_net.parameters(), lr=learning_rate)

In [21]:
#  Itera sobre el número de épocas definido anteriormente. En cada época, se entrena la red neuronal.
for epoch in range(num_epochs):
    
    # Se imprime un mensaje cada 5 épocas para mostrar el progreso del entrenamiento.
    if epoch % 5 == 0:
        print('Epoch {}'.format(epoch+1))
        
    # Los datos de entrenamiento (x_train y y_train) se mezclan utilizando la función shuffle de scikit-learn. 
    # Esto garantiza que los datos se presenten en un orden aleatorio durante el entrenamiento.    
    x_train, y_train = shuffle(x_train, y_train)
    # Mini batch learning
    
    # Itera sobre el número de lotes en cada época.
    for i in range(batch_no):
        
        # Se calcula el índice de inicio del lote actual.
        start = i * batch_size
        
        # Se calcula el índice de fin del lote actual.  
        end = start + batch_size
        
        # Se crea una variable x_var envolviendo el lote actual de características de entrenamiento en un tensor de punto flotante.
        x_var = Variable(torch.FloatTensor(x_train[start:end]))
        
        # Se crea una variable y_var envolviendo el lote actual de etiquetas de entrenamiento en un tensor de números enteros largos.
        y_var = Variable(torch.LongTensor(y_train[start:end]))
        
        # Se borran los gradientes acumulados de los parámetros del modelo antes de calcular los nuevos gradientes.
        optimizer.zero_grad()
        
        # Se realiza una pasada hacia adelante de los datos de entrada x_var a través de la red neuronal para obtener las predicciones ypred_var.
        ypred_var = new_net(x_var)
        
        # Se calcula la pérdida utilizando la función de pérdida criterion entre las predicciones ypred_var y las etiquetas y_var.
        loss =criterion(ypred_var, y_var)
        
        # Se realiza la retropropagación de la pérdida para calcular los gradientes de los parámetros de la red neuronal.
        loss.backward()
        
        # Se realiza un paso de optimización para actualizar los pesos de la red neuronal basados en los gradientes calculados.
        optimizer.step()

Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46
Epoch 51
Epoch 56
Epoch 61
Epoch 66
Epoch 71
Epoch 76
Epoch 81
Epoch 86
Epoch 91
Epoch 96


In [22]:
# Evaluate the model
test_var = Variable(torch.FloatTensor(x_val), requires_grad=True)
with torch.no_grad():
    result = new_net(test_var)
values, labels = torch.max(result, 1)
num_right = np.sum(labels.data.numpy() == y_val)
print('Accuracy {:.2f}'.format(num_right / len(y_val)))

Accuracy 0.83


In [23]:
# Applying model on the test data
X_test_var = Variable(torch.FloatTensor(X_test), requires_grad=True) 
with torch.no_grad():
    test_result = new_net(X_test_var)
values, labels = torch.max(test_result, 1)
survived = labels.data.numpy()

In [24]:
new_submission = [['PassengerId', 'Survived']]
for i in range(len(survived)):
    new_submission.append([X_test_passengers[i], survived[i]])

In [25]:
with open('new_submission.csv', 'w') as submissionFile:
    writer = csv.writer(submissionFile)
    writer.writerows(new_submission)
    
print('Writing Complete!')

Writing Complete!


In [26]:
new_submission = pd.read_csv('new_submission.csv')

In [27]:
new_submission = pd.DataFrame(new_submission)

In [28]:
new_submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [29]:
torch.save(new_net.state_dict(), 'new_net.pth')


In [30]:
X_train[100].shape

(19,)

In [31]:
# Ejemplo de datos de entrada
input_data = torch.tensor(X_train[10]).float()

# Asegúrate de que los datos de entrada tengan la forma correcta
input_data = input_data.view(1, -1)

predicted_labels = new_net.predict(input_data)

# Imprime las etiquetas predichas
print(predicted_labels)


[1]


In [32]:
# Supongamos que tienes un tensor input_data de tamaño (n_samples, n_features) con las muestras de entrada
input_data = torch.tensor(X_train[:10])  # Muestra 3

# Asegúrate de que los datos de entrada tengan la forma correcta
input_data = input_data.float()

# Obtén el número de muestras y características
n_samples, n_features = input_data.shape

# Cambia la forma de los datos de entrada para que sea (n_samples, n_features)
input_data = input_data.view(n_samples, n_features)

# Realiza las predicciones en el lote de muestras
predicted_labels = new_net.predict(input_data)

# Imprime las etiquetas predichas
print(predicted_labels)


[0 1 1 1 0 0 0 0 1 1]
